## 原生入侵種淡水魚分析
### Native Invasive Fresh Water fish Analysis

 style="text-align:right">特生中心 研究助理 曾子榮 

### 前言 

台灣的原生淡水魚中，有數種目前已藉由人為放流至新棲地，且成功建立大量族群，影響當地原有的魚類。然而由於這些入侵物種是原生種，而非外來種，而受到較少的關注。<br/>我手邊剛好有一份為了出版淡水魚紅皮書而整理的資料，或許可以從這份資料去抓出一些端倪，呈現這個現象、釐清各地區進來的時間、入侵後的趨勢、以及或許對原有群聚的影響。

### 分析概要 

1. 轉換成努力量
2. 使用二項式分佈
3. 推估可能的入侵時間
4. 視覺化


In [1]:
import pandas as pd
import pysal as ps
import matplotlib.path as mplPath
import numpy as np
import folium
from datetime import datetime
import os
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import time

Roses are <span style="color:red">red</span>, violets are <span style="color:blue">blue</span>.
Roses are <span style="color:red; font-family:Georgia; font-size:2em;">red.</span>

In [6]:
DATA = pd.read_excel('2017-02-08_re_out_ALLGPS_mod.xlsx')

### Remove wierd names (sea water fishes??)
Name = pd.read_excel('FreshWaterFishNames.xlsx')
N = Name.iloc[:,0].tolist()
BOOL = [i in N for i in DATA.Scientific_name]
DATA = DATA[BOOL]

Polygons = ps.open('淡水魚分佈區\\CatchmentArea6_WGS84.shp').read()

In [3]:
def PIPC(LAT,LON,polygon):
    '''
    Check whether a vector of points(Lat,Lon) is inside a polygon
    Return a vector of BOOL
    '''
    crd = np.array(polygon)# poly
    bbPath = mplPath.Path(crd)
    pnts = [[i,j] for i,j in zip(LON,LAT)] # points
    r = 0.001 # accuracy
    isIn = [ bbPath.contains_point(pnt,radius=r) or bbPath.contains_point(pnt,radius=-r) for pnt in pnts]
    return isIn

def TEC(polygon,fishname='Acrossocheilus paradoxus',after=2010,before=2020):
    '''
    Total effort calculator & fish appearance calculator
    '''
    BOOL = np.array(DATA.Year<=before) & np.array(DATA.Year>=after)
    TMP = DATA[BOOL]
    
    TMP3 = TMP.drop_duplicates(subset=['Latitude','Longitude','Year','Month','Source'],keep='first')
    TE = sum(PIPC(TMP3.Latitude,TMP3.Longitude,polygon))
    # Calculate fish appearance
    TMP4 = TMP[TMP.Scientific_name==fishname]
    FA = sum(PIPC(TMP4.Latitude,TMP4.Longitude,polygon))
    return (FA,TE)

In [4]:
def OUTTABLE(fishname='Acrossocheilus paradoxus',output=False):
    O = []
    for i,j in enumerate(['I','II','III','IV','V']):
        K = []
        K1 = []
        for Y in [[1800,1989],[1990,1995],[1996,2000],[2001,2002],[2003,2004],[2005,2006],[2007,2008],[2009,2010],[2011,2012],[2013,2020]]:
            TTT = TEC(Polygons[i].vertices,fishname=fishname,after = Y[0],before = Y[1])
            K.append(TTT[0])
            K1.append(TTT[1])
        O.append(pd.Series(K,name='R_'+j+'_FA'))
        O.append(pd.Series(K1,name='R_'+j+'_TE'))
    
    OUT = pd.DataFrame(O)
    OUT.columns = ['1800~1989','1990~1995','1996~2000','2001~2002','2003~2004','2005~2006','2007~2008','2009~2010','2011~2012','2013~']
    if output == True:        
        OUT.to_excel(str(datetime.now())[0:10]+'_'+fishname+'_Otable.xlsx')    
    print(fishname)
    return OUT

In [7]:
OUTTABLE('Acrossocheilus paradoxus')

Acrossocheilus paradoxus


,1800~1989,1990~1995,1996~2000,2001~2002,2003~2004,2005~2006,2007~2008,2009~2010,2011~2012,2013~
R_I_FA,38,18,132,63,421,281,33,99,7,36
R_I_TE,186,88,416,139,858,496,44,146,7,62
R_II_FA,11,33,122,140,467,214,184,195,131,34
R_II_TE,40,90,303,562,1151,594,365,439,181,55
R_III_FA,9,9,97,119,125,109,43,9,50,5
R_III_TE,128,49,654,503,586,419,167,145,225,141
R_IV_FA,0,0,0,0,3,0,0,1,26,0
R_IV_TE,23,11,47,70,136,1,10,29,35,4
R_V_FA,0,0,0,24,188,143,0,2,4,3
R_V_TE,30,40,11,54,529,342,6,51,41,111


In [8]:
OUTTABLE('Spinibarbus hollandi')

Spinibarbus hollandi


,1800~1989,1990~1995,1996~2000,2001~2002,2003~2004,2005~2006,2007~2008,2009~2010,2011~2012,2013~
R_I_FA,0,0,0,0,0,0,0,1,0,0
R_I_TE,186,88,416,139,858,496,44,146,7,62
R_II_FA,0,0,0,0,6,3,1,3,6,0
R_II_TE,40,90,303,562,1151,594,365,439,181,55
R_III_FA,3,0,1,13,13,24,11,13,15,2
R_III_TE,128,49,654,503,586,419,167,145,225,141
R_IV_FA,0,0,0,0,0,0,0,0,0,0
R_IV_TE,23,11,47,70,136,1,10,29,35,4
R_V_FA,1,2,3,9,46,38,0,9,4,11
R_V_TE,30,40,11,54,529,342,6,51,41,111


In [9]:
OUTTABLE('Opsariichthys pachycephalus')

Opsariichthys pachycephalus


,1800~1989,1990~1995,1996~2000,2001~2002,2003~2004,2005~2006,2007~2008,2009~2010,2011~2012,2013~
R_I_FA,21,31,85,28,301,180,2,40,5,6
R_I_TE,186,88,416,139,858,496,44,146,7,62
R_II_FA,11,22,90,221,630,342,184,242,116,19
R_II_TE,40,90,303,562,1151,594,365,439,181,55
R_III_FA,2,4,204,200,260,134,83,56,110,67
R_III_TE,128,49,654,503,586,419,167,145,225,141
R_IV_FA,0,0,0,31,36,0,0,0,27,0
R_IV_TE,23,11,47,70,136,1,10,29,35,4
R_V_FA,0,0,1,36,257,176,0,6,15,22
R_V_TE,30,40,11,54,529,342,6,51,41,111


In [10]:
OUTTABLE('Rhinogobius candidianus')

Rhinogobius candidianus


,1800~1989,1990~1995,1996~2000,2001~2002,2003~2004,2005~2006,2007~2008,2009~2010,2011~2012,2013~
R_I_FA,0,5,186,47,384,314,12,88,10,16
R_I_TE,186,88,416,139,858,496,44,146,7,62
R_II_FA,0,18,151,389,701,385,243,309,174,31
R_II_TE,40,90,303,562,1151,594,365,439,181,55
R_III_FA,0,0,220,170,227,176,106,73,103,52
R_III_TE,128,49,654,503,586,419,167,145,225,141
R_IV_FA,0,0,0,6,2,0,0,0,15,0
R_IV_TE,23,11,47,70,136,1,10,29,35,4
R_V_FA,0,0,0,18,102,44,0,1,8,0
R_V_TE,30,40,11,54,529,342,6,51,41,111


In [3]:
## I can change the css style based on web references by the folowing code

# url_1:http://kevinburke.bitbucket.org/markdowncss/markdown.css
# url_2:https://raw.githubusercontent.com/titipata/klab_ipython_notebook/master/custom.css
# import requests
# from IPython.core.display import HTML
# link = requests.get("https://raw.githubusercontent.com/titipata/klab_ipython_notebook/master/custom.css")
# HTML("<style>"+link.content.decode()+"</style>")